In [ ]:
import tensorflow as tf
import numpy as np

def compute_interior_loss(batch_data_train, batch_labels_train, NN,model_constraint,label_dimensions):
        '''
        compute the weak form loss - problem specific
        '''
        forcing=tf.constant([1.0])
        # break apart input for tracking (to compute gradient)
        x = tf.convert_to_tensor(batch_data_train[:,0:1])
        y = tf.convert_to_tensor(batch_data_train[:,1:2])

        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            tape.watch(y)

            # restack to feed through model
            batch = tf.stack([x[:,0], y[:,0]], axis=1)

            # make prediction
            pred = NN(batch)

            # compute gradients wrt input
            dx = tape.gradient(pred, x)
            dy = tape.gradient(pred, y)

        # second derivatives
        ddx = tape.gradient(dx, x)
        ddy = tape.gradient(dy, y)
        
        laplace_u = ddx + ddy
        del tape
        
        # pointwise strong form residual
        pde_res = laplace_u + forcing

        interior_loss = tf.reduce_mean(tf.square(pde_res))
        return model_constraint*interior_loss
